In [ ]:
import MeCab
mecab = MeCab.Tagger ('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

In [ ]:
from wordcloud import WordCloud

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

In [ ]:
DATAFOLDER = '../../data_handson/WordCloud/'

In [ ]:
path = DATAFOLDER + "urls_au_claim.txt"
with open(path) as f:
    urls = [s.strip() for s in f.readlines()]

In [ ]:
ids = []
timestamps = []
names = []
messages = []
titles = []

for url in urls:
    text = requests.get(url).text
    soup = bs(text, "html5lib")

    title_tag =  soup.find_all("h1", {"class":"title"})
    if len(title_tag) > 0:
        title = title_tag[0].text.rstrip('\n')
    else:
        title = '-'
        
    posts =soup.find_all('div', {'class':"post"})
    num_post = len(posts)
    if num_post > 0:
        for i in range(num_post):
            ids.append(int(posts[i]['data-id']))
            names.append(posts[i].find('span', {'class':'name'}).text)
            timestamps.append(posts[i].find('span', {'class':'date'}).text)
            messages.append(posts[i].find_all(('div', {'class':'message'}))[1].text)
            titles.append(title)


In [ ]:
df = pd.DataFrame([ids,
                   timestamps,
                   names,
                   messages,
                  titles]).T
df.columns = ["id", 'timestamp','name','message','thread']

In [ ]:
df['thread'].value_counts()

In [ ]:
df.to_csv(DATAFOLDER + '{}.csv'.format(titles[0]), index=False)

In [ ]:
def div_text(text):
    pattern = '(http|https)://(([\w-]+\.?)+/)+'
    text = re.sub(pattern, '', text)
    mecab.parse('')
    node = mecab.parseToNode(text)
    output = []
    while node:
            if node.surface != "": 
                word_type = node.feature.split(",")[0]
                if word_type in ["形容詞", "動詞","名詞"]:
#                 if word_type in ["形容詞", "動詞","名詞", "副詞"]:
                    output.append(node.surface)
            node = node.next
            if node is None:
                break
    return output

In [ ]:
texts = df['message'].apply(div_text)
word_list = []
for i in range(len(texts)):
    word_list.extend(texts[i])
words = ''
del_list = [ 'http://','http','https', 'www', 'rio', '5ch', '.net', 'test', 'read', 'cgi', 'yuu', 'se','無断転載禁止','禁止','無断転載']
for w in word_list:
    if w not in del_list:
        words += w + ' '

In [ ]:
def show_wordcloud(words):
    
    font_path="NotoSansCJKjp-Regular.otf"

    # ストップワードの設定
    stop_words = [ 'てる','いる','なる','れる','する','ある','こと','さん',' それ','もん','みたい',\
                  'よう','これ','ない','なり','しろ','なれ','そう','なん','もう','なら','やる','ため','られ',\
                 '思う','まだ','どんどん','それ','られる','なっ','思っ',]

    wordcloud = WordCloud(background_color="white",
                          font_path=font_path, 
                          width=900,
                          height=500, 
                          stopwords=set(stop_words)).generate(words)

    plt.figure(figsize=(15,12))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.savefig(DATAFOLDER + '{}.png'.format(titles[0]))
    plt.show()

In [ ]:
show_wordcloud(words)